# Topic Modeling w LDA (Used Gensim)

Dataset Available - http://qwone.com/~jason/20Newsgroups/

First, fix the verbosity of the logger. In this example we're logging only warnings, but for a better debug, uprint all the INFOs.

In [1]:
import logging
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.WARNING)
logging.root.level = logging.WARNING

In [2]:
from sklearn import datasets
news_dataset = datasets.fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))

In [14]:
#news_dataset
#news_dataset.data

We do now have a collection of documents.

In [7]:
# A list of text document is contained in the data variable
documents = news_dataset.data

print "In the dataset there are", len(documents), "textual documents"
print "And this is the first one:\n", documents[0]

In the dataset there are 18846 textual documents
And this is the first one:


I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about the recent Pens massacre of the Devils. Actually,
I am  bit puzzled too and a bit relieved. However, I am going to put an end
to non-PIttsburghers' relief with a bit of praise for the Pens. Man, they
are killing those Devils worse than I thought. Jagr just showed you why
he is much better than his regular season stats. He is also a lot
fo fun to watch in the playoffs. Bowman should let JAgr have a lot of
fun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final
regular season game.          PENS RULE!!!




At first, we're gonna import all the modules we need.

In [9]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

Then, we define a word tokenizer (https://en.wikipedia.org/wiki/Tokenization_(lexical_analysis)) with stopword removal (common words like "the", "are" and "and" are excuded from the processing, since they don't have discriminative power and they just increase the processing complexity).

In [10]:
def tokenize(text):
    return [token for token in gensim.utils.simple_preprocess(text) if token not in gensim.parsing.preprocessing.STOPWORDS]

print "After the tokenizer, the previous document becomes:\n", tokenize(documents[0])

After the tokenizer, the previous document becomes:
[u'sure', u'bashers', u'pens', u'fans', u'pretty', u'confused', u'lack', u'kind', u'posts', u'recent', u'pens', u'massacre', u'devils', u'actually', u'bit', u'puzzled', u'bit', u'relieved', u'going', u'end', u'non', u'pittsburghers', u'relief', u'bit', u'praise', u'pens', u'man', u'killing', u'devils', u'worse', u'thought', u'jagr', u'showed', u'better', u'regular', u'season', u'stats', u'lot', u'fo', u'fun', u'watch', u'playoffs', u'bowman', u'let', u'jagr', u'lot', u'fun', u'couple', u'games', u'pens', u'going', u'beat', u'pulp', u'jersey', u'disappointed', u'islanders', u'lose', u'final', u'regular', u'season', u'game', u'pens', u'rule']


Next step: tokenise all the documents and build a count dictionary, that contains the count of the tokens over the complete text corpus.

In [15]:
processed_docs = [tokenize(doc) for doc in documents]
word_count_dict = gensim.corpora.Dictionary(processed_docs)
print "In the corpus there are", len(word_count_dict), "unique tokens"

In the corpus there are 95507 unique tokens


We might want to further lower the complexity of the process, removing all the very rare tokens (the ones appearing in less than 20 documents) and the very popular ones (the ones appearing in more than 20% documents; in our case circa 4'000)

In [17]:
word_count_dict.filter_extremes(no_below=20, no_above=0.1) # word must appear >10 times, and no more than 20% documents
print "After filtering, in the corpus there are only", len(word_count_dict), "unique tokens"

After filtering, in the corpus there are only 8121 unique tokens


Let's not build the bag of words representation (https://en.wikipedia.org/wiki/Bag-of-words_model) of the text documents, to create a nice vector space model (https://en.wikipedia.org/wiki/Vector_space_model). Within this methaphor, a vector lists the multiplicity of the tokens appearing in the document. The vector is indexed by the dictionary of tokens, previously built. Note that, since a restricted subset of words appears in each document, this vector is often represented in a sparse way.

In [18]:
bag_of_words_corpus = [word_count_dict.doc2bow(pdoc) for pdoc in processed_docs]

In [19]:
bow_doc1 = bag_of_words_corpus[0]

print "Bag of words representation of the first document (tuples are composed by token_id and multiplicity):\n", bow_doc1
print
for i in range(5):
    print "In the document, topic_id {} (word \"{}\") appears {} time[s]".format(bow_doc1[i][0], word_count_dict[bow_doc1[i][0]], bow_doc1[i][1])
print "..."

Bag of words representation of the first document (tuples are composed by token_id and multiplicity):
[(213, 1), (760, 2), (771, 2), (1345, 1), (1367, 1), (1710, 2), (2008, 1), (2680, 1), (3162, 1), (3183, 1), (3321, 1), (3436, 1), (3727, 1), (3825, 1), (3850, 1), (3936, 1), (4180, 1), (4271, 2), (4639, 1), (4664, 1), (4801, 1), (4856, 1), (4961, 1), (5204, 5), (5360, 2), (5367, 1), (5415, 2), (5472, 3), (5658, 1), (5845, 1), (5953, 1), (6180, 1), (6241, 1), (6361, 1), (6405, 1), (6647, 1), (6823, 2), (6857, 1), (7007, 1), (7140, 1), (7164, 1), (7348, 1), (7878, 1), (7964, 1)]

In the document, topic_id 213 (word "showed") appears 1 time[s]
In the document, topic_id 760 (word "jagr") appears 2 time[s]
In the document, topic_id 771 (word "going") appears 2 time[s]
In the document, topic_id 1345 (word "recent") appears 1 time[s]
In the document, topic_id 1367 (word "couple") appears 1 time[s]
...


Now, finally, the core algorithm of the analysis: LDA. Gensim offers two implementations: a monocore one, and a multicore. We use the monocore one, setting the number of topics equal to 10 (you can change it, and check the results). Try the multicore to prove the speedup!

In [20]:
# LDA mono-core
lda_model = gensim.models.LdaModel(bag_of_words_corpus, num_topics=10, id2word=word_count_dict, passes=5)

# LDA multicore (in this configuration, defaulty, uses n_cores-1)
# lda_model = gensim.models.LdaMulticore(bag_of_words_corpus, num_topics=10, id2word=word_count_dict, passes=5)

Here's a list of the words (and their relative weights) for each topic:

In [21]:
_ = lda_model.print_topics(-1)

Let's print now the topics composition, and their scores, for the first document. You will see that only few topics are represented; the others have a nil score.

In [22]:
for index, score in sorted(lda_model[bag_of_words_corpus[0]], key=lambda tup: -1*tup[1]):
    print "Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 10))

Score: 0.744185814751	 Topic: 0.020*game + 0.014*team + 0.012*games + 0.009*play + 0.008*hockey + 0.008*season + 0.007*st + 0.007*year + 0.006*league + 0.006*win
Score: 0.11369374105	 Topic: 0.004*problem + 0.004*medical + 0.004*evidence + 0.003*science + 0.003*case + 0.003*point + 0.003*cause + 0.003*cases + 0.003*number + 0.003*drug
Score: 0.076324941638	 Topic: 0.022*image + 0.016*bit + 0.014*card + 0.013*color + 0.011*video + 0.009*jpeg + 0.008*images + 0.007*monitor + 0.007*gif + 0.007*mb
Score: 0.0554478229735	 Topic: 0.008*government + 0.008*said + 0.006*gun + 0.005*armenian + 0.005*president + 0.005*mr + 0.005*israel + 0.005*war + 0.005*state + 0.005*armenians


That's wonderful! LDA is able to understand that the article is about a team game, hockey, even though the work hockey never appears in the document. Checking the ground truth for that document (the newsgroup category) it's actually correct! It was posted in sport/hockey category. Other topics, if any, account for less than 5%, so they have to be considered marginals (dirt).

In [23]:
news_dataset.target_names[news_dataset.target[0]]

'rec.sport.hockey'

So far, we have dealt with documents contained in the training set. What if we need to process an unseed document? Fortunately, we don't need to re-train the system (wasting lots of time), as we can just infer its topics.

# Model Accuracy Measure

In [25]:
print "Log perplexity of the model is", lda_model.log_perplexity(bag_of_words_corpus)

Log perplexity of the model is -7.58679426525


# Test Data - Unseen Document

In [24]:
unseen_document = "In my spare time I either play badmington or drive my car"
print "The unseen document is composed by the following text:", unseen_document
print

bow_vector = word_count_dict.doc2bow(tokenize(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print "Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5))

The unseen document is composed by the following text: In my spare time I either play badmington or drive my car

Score: 0.656124494095	 Topic: 0.009*drive + 0.007*car + 0.006*problem + 0.005*thanks + 0.005*hard
Score: 0.18387411826	 Topic: 0.020*game + 0.014*team + 0.012*games + 0.009*play + 0.008*hockey
Score: 0.0200003945828	 Topic: 0.017*god + 0.007*believe + 0.006*jesus + 0.004*things + 0.004*bible
Score: 0.0200003192788	 Topic: 0.019*key + 0.009*public + 0.009*information + 0.008*government + 0.008*encryption
Score: 0.0200002326032	 Topic: 0.008*government + 0.008*said + 0.006*gun + 0.005*armenian + 0.005*president
Score: 0.0200002289037	 Topic: 0.012*space + 0.009*year + 0.005*years + 0.005*earth + 0.005*nasa
Score: 0.020000128039	 Topic: 0.004*problem + 0.004*medical + 0.004*evidence + 0.003*science + 0.003*case
Score: 0.0200000728008	 Topic: 0.013*edu + 0.011*file + 0.011*windows + 0.009*dos + 0.008*com
Score: 0.0200000114351	 Topic: 0.022*image + 0.016*bit + 0.014*card + 0.01

# Reference

1. http://nbviewer.ipython.org/gist/boskaiolo/cc3e1341f59bfbd02726
2. http://engineering.intenthq.com/2015/02/automatic-topic-modelling-with-lda/